In [ ]:
import numpy as np
import pandas as pd
import re
import warnings
import plotly.express as px
import plotly.graph_objects as go
warnings.filterwarnings('ignore')

In [ ]:
DF = pd.read_csv('final_flats_data.csv', index_col = 'Unnamed: 0')

In [ ]:
DF.head()

In [ ]:
def prepare_df(df):
    df = df.copy()
    df['Количество комнат'] = df['Количество комнат'].map(lambda x: re.sub(
        '[c, с]тудия', '0.5', x.replace('> 9', '10')) if type(x) == str else x).astype(float)
    df['Количество комнат'] = df['Количество комнат'].where(df['Количество комнат'] < 10, 10)
    df['Округ'] = df['Округ'].map(lambda s: ''.join([x[0].upper() for x in re.split('-| ', s)]))
    df["Цена за м2"] = df['Цена']/df['Площадь']
    return df
new_DF = prepare_df(DF)                             

In [ ]:
def plot_bar(df, col, stat, groupby, hue = None, fig = 'bar', orientation = 'v', w = 1000, h = 600, sort = True):
    df = df.copy()
    if hue == 'год' or groupby == 'год':
        year = pd.to_datetime(df['Время загрузки']).dt.year
        year.name = 'год'
        df = pd.concat([df, year], axis = 1)
    if hue == 'месяц' or groupby == 'месяц':
        month = pd.to_datetime(df['Время загрузки']).dt.strftime('%Y-%m')
        month.name = 'месяц'
        df = pd.concat([df, month], axis = 1)
    
    if hue is None:
        gr_cols = groupby
    else:
        gr_cols = [groupby, hue]
    for_bar = df.groupby(gr_cols, as_index = False).agg({col: stat}).astype({groupby: str})
    if sort:
        for_bar = for_bar.sort_values(col)
    #display(for_bar)
    if orientation == 'v':
        x = groupby 
        y = col
    else:
        y = groupby 
        x = col
    if fig == 'bar':
        fig = px.histogram(for_bar, x = x, y = y, color = hue, barmode = 'group', 
                 orientation = orientation, width = w, height = h)
    elif fig == 'scatter':
        fig = px.scatter(for_bar, x = x, y = y, color = hue, width = w, height = h)
    elif fig == 'line':
        fig = px.line(for_bar, x = x, y = y, color = hue, width = w, height = h, markers=True)
    fig.show()

In [ ]:
plot_bar(new_DF, 'Цена', 'median', 'Округ', 'год')

In [ ]:
plot_bar(new_DF, 'Цена', 'median', 'Количество комнат', 'год')

In [ ]:
plot_bar(new_DF, 'Цена', 'count', 'Количество комнат')

In [ ]:
plot_bar(new_DF, 'Цена', 'median', 'Район', None, orientation = 'h', h = 1500)

In [ ]:
plot_bar(new_DF, 'Цена', 'median', 'Ветка', None, orientation = 'h', h = 700)

In [ ]:
plot_bar(new_DF, 'Цена', 'median', 'Ремонт')

In [ ]:
plot_bar(new_DF, 'Цена', 'median', 'месяц', 'Округ', fig = 'line', sort = False)

In [ ]:
plot_bar(new_DF, 'Цена', 'median', 'месяц', 'Количество комнат', fig = 'line', sort = False)

In [ ]:
plot_bar(new_DF, "Цена за м2" , 'mean', 'месяц', 'Количество комнат', fig = 'line', sort = False)

In [ ]:
plot_bar(new_DF, "Цена за м2" ,'median', 'месяц', fig = 'line', sort = False)

In [ ]:
plot_bar(new_DF, "Цена за м2" ,'mean', 'месяц', fig = 'line', sort = False)

In [ ]:
def clear_anomal(df, APERTURE_SIZE = 6):
    
    df = df.drop_duplicates(subset = ['Cсылка', 'Цена', 'Адрес']).drop_duplicates(subset = ['ID', 'Цена', 'Адрес'])
    df = df[df['Количество комнат'].isin([*range(1, 7), 0.5])]
    quantiles = df.groupby('Количество комнат', as_index = False).agg({'Цена': [lambda df: df.quantile(0.98), 
    lambda df: df.quantile(0.01)], 'Площадь': [lambda df: df.quantile(0.99), lambda df: df.quantile(0.01)]})
    quantiles.columns = ['Количество комнат', 'price_q_r', 'price_q_l', 'S_q_r', 'S_q_l']
    
    df = df.merge(quantiles, on = 'Количество комнат')
    df = df[(df['Цена'] > df['price_q_l'])&(df['Цена'] < df['price_q_r'])]
    df = df[(df['Площадь'] > df['S_q_l'])&(df['Площадь'] < df['S_q_r'])]
    df = df[~((df['Цена'] >= df[df['Количество комнат'] == 3]['Цена'].quantile(0.95))&(df['Количество комнат'] == 3))]
    df = df[~((df['Площадь'] >= df[df['Количество комнат'] == 0.5]['Площадь'].quantile(0.98))&(df['Количество комнат'] == 0.5))]
    df = df[~((df['Площадь'] >= df[df['Количество комнат'] == 2]['Площадь'].quantile(0.98))&(df['Количество комнат'] == 2))]
    df = df[df['Цена'] < df['Цена'].quantile(0.97)]
    df = df[df['Площадь'] < df['Площадь'].quantile(0.97)]
    
    df['hex_id'] = df.apply(lambda x: h3.geo_to_h3(x.lat, x.lng, APERTURE_SIZE), 1)
    top30hex = set(df['hex_id'].value_counts().nlargest(30).index)
    df['top30hex'] = df['hex_id'].map(lambda x: x if x in top30hex else -1)
    return df
    
clean_DF = clear_anomal(new_DF) 

In [ ]:
clean_DF.to_csv('clean_flats_df.csv', index=False)

In [ ]:
fig = px.histogram(clean_DF, x="Цена", color='Количество комнат')
fig.show()

In [ ]:
fig = px.histogram(clean_DF, x='Площадь', color='Количество комнат')
fig.show()

In [ ]:
fig = px.box(clean_DF, x="Цена", color='Количество комнат', height = 800)
fig.show()

In [ ]:
fig = px.box(clean_DF, x="Площадь", color='Количество комнат', height = 800)
fig.show()

In [ ]:
import folium

def show_circles_on_map(data, latitude_column, longitude_column, color):
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    
    data: DataFrame that contains columns latitude_column and longitude_column
    latitude_column: string, the name of column for latitude coordinates
    longitude_column: string, the name of column for longitude coordinates
    color: string, the color of circles to be drawn
    """

    location = (data[latitude_column].mean(), data[longitude_column].mean())
    m = folium.Map(location=location)

    for _, row in data.iterrows():
        folium.Circle(
            radius=100,
            location=(row[latitude_column], row[longitude_column]),
            color=color,
            fill_color=color,
            fill=True
        ).add_to(m)

    return m

In [ ]:
show_circles_on_map(clean_DF.sample(1000), 'lat', 'lng', 'blue')

In [ ]:
clean_DF

In [ ]:
import h3

geo_DF = clean_DF[['hex_id', 'Цена', 'Площадь', 'lat', 'lng', "Цена за м2"]]

# find hexs containing the points
geo_DF = geo_DF.groupby('hex_id', as_index = False).agg({'Цена': ['median', 'mean', 'count'], 
                                    'Площадь': 'mean', "Цена за м2": ['median', 'mean']})
geo_DF.columns = [' '.join(col).strip() for col in geo_DF.columns.values]
geo_DF.head()

In [ ]:
from geojson import Feature, Point, FeatureCollection
import json

def hexagons_dataframe_to_geojson(df_hex, file_output = None, column_name = "value", tool_f = None):
    """
    Produce the GeoJSON for a dataframe, constructing the geometry from the "hex_id" column
    and with a property matching the one in column_name
    """    
    list_features = []
    
    for i,row in df_hex.iterrows():
        try:
            geometry_for_row = { "type" : "Polygon", "coordinates": [h3.h3_to_geo_boundary(h=row["hex_id"],geo_json=True)]}
            if tool_f is not None:
                properties = {column_name : row[column_name], tool_f[0]: tool_f[1](row[column_name])}
            else:
                properties = {column_name : row[column_name]}
            feature = Feature(geometry = geometry_for_row , id=row["hex_id"], properties = properties)
            list_features.append(feature)
        except:
            print("An exception occurred for hex " + row["hex_id"]) 

    feat_collection = FeatureCollection(list_features)
    geojson_result = json.dumps(feat_collection)
    return geojson_result

def get_color(custom_cm, val, vmin, vmax, color_f = lambda x: x**0.3):
    if custom_cm.name == 'hsv':
        return matplotlib.colors.to_hex(custom_cm(0.6 - 0.6*(color_f(val)-color_f(vmin))/(color_f(vmax)-color_f(vmin))))
    else:
        return matplotlib.colors.to_hex(custom_cm((val-vmin)/(vmax-vmin)))

def choropleth_map(df_aggreg, column_name = "value", border_color = 'black', fill_opacity = 0.7, 
                   color_map_name = "Blues", initial_map = None, tool_f = None):
    """
    Creates choropleth maps given the aggregated data. initial_map can be an existing map to draw on top of.
    """    
    #colormap
    min_value = df_aggreg[column_name].min()
    max_value = df_aggreg[column_name].max()
    mean_value = df_aggreg[column_name].mean()
    print(f"Colour column min value {min_value}, max value {max_value}, mean value {mean_value}")
    print(f"Hexagon cell count: {df_aggreg['hex_id'].nunique()}")
    
    # the name of the layer just needs to be unique, put something silly there for now:
    name_layer = "Choropleth " + str(df_aggreg)
    
    if initial_map is None:
        initial_map = folium.Map(location= (55.7522200, 37.6155600), zoom_start=10)#, tiles="cartodbpositron")

    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg, column_name = column_name, tool_f = tool_f)

    # color_map_name 'Blues' for now, many more at https://matplotlib.org/stable/tutorials/colors/colormaps.html to choose from!
    custom_cm = matplotlib.cm.get_cmap(color_map_name)

    folium.features.GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': get_color(custom_cm, feature['properties'][column_name], vmin=min_value, vmax=max_value),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity
            #'dashArray': ' '.join([str(feature['properties'][column_name]/10**6), 'млн. руб.'])
        }, 
        highlight_function = lambda feature: {
              'opacity' : 0.90,
                  # specifying properties from GeoJSON
              #'weight' : x['properties']['stroke-width'],
              #'dashArray' : ' '.join([str(feature['properties'][column_name]/10**6), 'млн. руб.'])
        },
        tooltip=folium.features.GeoJsonTooltip(
        # using fields from the geojson file
        fields=[column_name if tool_f is None else tool_f[0]],
        aliases=[f'{column_name if tool_f is None else tool_f[0]}: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") # setting style for popup box
        ),
        name = name_layer, 
    ).add_to(initial_map)

    return initial_map

In [ ]:
import matplotlib
choropleth_map(geo_DF, 'Цена median', color_map_name = 'hsv', fill_opacity = 0.5,
               tool_f = ('Медиана цены', lambda x: ' '.join([str(x/10**6), 'млн. руб.'])))

In [ ]:
choropleth_map(geo_DF, 'Цена mean', color_map_name = 'hsv', fill_opacity = 0.5,
               tool_f = ('Средняя цена', lambda x: ' '.join([str(round(x/10**6, 2)), 'млн. руб.'])))

In [ ]:
choropleth_map(geo_DF, 'Цена за м2 mean', color_map_name = 'hsv', fill_opacity = 0.5,
               tool_f = ('Средняя цена м2', lambda x: ' '.join([str(round(x/10**3, 2)), 'тыс. руб.'])))

In [ ]:
choropleth_map(geo_DF, 'Цена count', color_map_name = 'hsv', fill_opacity = 0.5,
               tool_f = ('Количество квартир', lambda x: ' '.join([str(round(x/10**3, 2)), 'тыс.'])))

In [ ]:
def get_stat_by_room(df, room):
    df = df[['hex_id', 'Количество комнат', 'Цена', 'Площадь', 
             'lat', 'lng', "Цена за м2"]][df['Количество комнат'] == room]
    df = df.groupby(['Количество комнат', 'hex_id'], as_index = False).agg({'Цена': ['median', 'mean', 'count'], 
                               'Площадь': 'mean', "Цена за м2": ['median', 'mean']})
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    return df
    
dict_DF = {}

for room in [*range(1,7), 0.5]:
    dict_DF[room] = get_stat_by_room(clean_DF, room) 


In [ ]:
choropleth_map(dict_DF[3], 'Цена mean', color_map_name = 'hsv', fill_opacity = 0.5,
               tool_f = ('Средняя цена', lambda x: ' '.join([str(round(x/10**6, 2)), 'млн. руб.'])))